In [1]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories
from pathlib import Path

In [6]:
params = read_yaml(PARAMS_FILE_PATH)

[2024-04-03 13:09:25,356: INFO: common: yaml file: params.yaml loaded successfully]


In [7]:
params.OFFSET

-1

In [8]:
import os
from subprocess import Popen, PIPE
from pathlib import Path

folderToCheck = Path('artifacts/datasets/farm_copilot_dataset')

def checkImage(fn):
  proc = Popen(['identify', '-verbose', fn], stdout=PIPE, stderr=PIPE)
  out, err = proc.communicate()
  exitcode = proc.returncode
  return exitcode, out, err

for directory, subdirectories, files, in os.walk(folderToCheck):
  for file in files:
      filePath = os.path.join(directory, file)
      code, output, error = checkImage(filePath)
      if str(code) !="0" or str(error, "utf-8") != "":
        print("ERROR " + filePath)
      else:
        print("OK " + filePath)

print("-------------- DONE --------------");

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [24]:
def is_valid_jpeg(image_path):
    try:
        img = Image.open(image_path)
        img.verify()
        return True
    except (IOError, SyntaxError):
        return False

In [25]:
from PIL import Image
def resize_image(input_image_path, output_image_path, size=(224,224)):
    original_image = Image.open(input_image_path)
    resize_image = original_image.resize(size)
    resize_image.save(output_image_path)

In [27]:
import os
from pathlib import Path

folderToCheck = Path('./artifacts/datasets/farm_copilot_dataset_150')


valid_ext = ['.JPG', '.PNG', '.GIF', '.BMP', '.jpg', '.png', '.gif', '.bmp']


for (root, dir, files) in os.walk(folderToCheck):
  for file in files:
    filename, ext = os.path.splitext(p=os.path.join(root, file))
    if ext not in valid_ext:
        os.remove(os.path.join(root, file))
        print(f"{file} with extension {ext} removed. Reason: Invalid extension")
        continue
    
    image = os.path.join(root,file)
    if not is_valid_jpeg(image_path=image):
        os.remove(image)
        print(f"{image} removed. Reason: Corrupted")
        continue
    
    try:
        resize_image(input_image_path=image, output_image_path=image)
    except (IOError, SyntaxError):
        os.remove(image)
        print(f"{image} removed. Reason: Failed to resize")

artifacts\datasets\farm_copilot_dataset_150\plant_corn-healthy\healthy189_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-healthy\healthy87_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight379_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight37_.jpg removed. Reason: Corrupted
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight452_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight453_.jpg removed. Reason: Corrupted
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight522_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-blight\leaf blight58_.jpg removed. Reason: Failed to resize
artifacts\datasets\farm_copilot_dataset_150\plant_corn-leaf-bligh

In [3]:
from tensorflow import keras
from pathlib import Path


datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        brightness_range=[0.2, 1.5],
        horizontal_flip=True,
        fill_mode='nearest')

def augment_data(files,root, dir, num_of_gen_img: int, save_to: Path = Path('../artifacts/datasets/farm_copilot_dataset_150')):
    
    for file in files:
        path_to_file = os.path.join(root, file)
        prefix = root.split('\\')[-1]

        _, ext = os.path.splitext(p=path_to_file)
        print(f"root: {root}")
        img = keras.preprocessing.image.load_img(path_to_file)  # this is a PIL image
        x = keras.preprocessing.image.img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=os.path.join(save_to, prefix), save_prefix=prefix, save_format=ext):
            i += 1
            if i > num_of_gen_img:
                break  # otherwise the generator would loop indefinitely

In [7]:
import os
from pathlib import Path

folderToCheck = Path('../artifacts/datasets/farm_copilot_dataset')


for (root, dir, files) in os.walk(folderToCheck):
  if len(files) <= 300:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=100) 
  elif len(files) > 300 and len(files) <= 400:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=75)
  elif len(files) > 400 and len(files) <= 500:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=60)  
  elif len(files) > 500 and len(files) <= 600:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=50)  
  elif len(files) > 600 and len(files) <= 700:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=43)
  elif len(files) > 700 and len(files) <= 800:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=38)
  elif len(files) > 800 and len(files) <= 900:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=34) 
  elif len(files) > 900 and len(files) <= 1000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=30)
  elif len(files) > 1000 and len(files) <= 2000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=15)
  elif len(files) > 2000 and len(files) <= 3000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=10)  
  elif len(files) > 3000 and len(files) <= 4000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=8) 
  elif len(files) > 4000 and len(files) <= 5000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=6)
  elif len(files) > 5000 and len(files) <= 7000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=5) 
  elif len(files) > 7000 and len(files) <= 9000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=4)
  elif len(files) > 400 and len(files) <= 10000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=3) 
  elif len(files) > 10000 and len(files) <= 15000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=2) 
  elif len(files) > 15000 and len(files) <= 20000:
    augment_data(files=files, dir=dir, root=root, num_of_gen_img=1)                

root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_copilot_dataset\plant_pest_bollworm
root: ..\artifacts\datasets\farm_c

In [ ]:
import os
from pathlib import Path

folderToCheck = Path('../artifacts/datasets/farm_copilot_dataset')


for (root, dir, files) in os.walk(folderToCheck):
    for file in files:
        filename, ext = os.path.splitext(p=os.path.join(root,file))
        prefix = root.split('\\')[-1]
        print(f"root: {root}\ndir:{dir}\nfile:{file}\nFilename:{filename}\nExtension:{ext}\nPrefix:{prefix}")
        break
    

root: ..\artifacts\datasets\farm_copilot_dataset\plant_apple-black-rot
dir:[]
file:0090d05d-d797-4c99-abd4-3b9cb323a5fd___JR_FrgE.S 8727.JPG
Filename:..\artifacts\datasets\farm_copilot_dataset\plant_apple-black-rot\0090d05d-d797-4c99-abd4-3b9cb323a5fd___JR_FrgE.S 8727
Extension:.JPG
Prefix:plant_apple-black-rot
root: ..\artifacts\datasets\farm_copilot_dataset\plant_apple_apple-scab
dir:[]
file:00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG
Filename:..\artifacts\datasets\farm_copilot_dataset\plant_apple_apple-scab\00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335
Extension:.JPG
Prefix:plant_apple_apple-scab
root: ..\artifacts\datasets\farm_copilot_dataset\plant_apple_cedar-apple-rust
dir:[]
file:025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655.JPG
Filename:..\artifacts\datasets\farm_copilot_dataset\plant_apple_cedar-apple-rust\025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655
Extension:.JPG
Prefix:plant_apple_cedar-apple-rust
root: ..\artifacts\datasets\farm_

In [ ]:
import tensorflow as tf 

In [3]:
import splitfolders
splitfolders.ratio('../artifacts/datasets/farm_copilot_dataset_150', output="../artifacts/datasets/", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 622326 files [39:44, 261.02 files/s]
